In [5]:
from dataclasses import dataclass
from typing import List, Tuple

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast, AutoTokenizer
from datasets import load_dataset
from transformers import RobertaTokenizer

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
# Global variables
VAL_DATASET_LENGTH = 200
USE_SMALL_DATASET = True

In [32]:
# load dataset from datasets/strategyqa_train_filtered.json
dataset = load_dataset("json", data_files={"train": "../datasets/strategyqa_train_filtered.json", "test": "../datasets/strategyqa_test.json"})
print(dataset)
# initialize training, validation, and testing dataset
train_dataset = dataset['train'].select(indices=range(len(dataset['train']) - VAL_DATASET_LENGTH))
val_dataset = dataset['train'].select(indices=range(len(dataset['train']) - VAL_DATASET_LENGTH, len(dataset['train'])))
test_dataset = dataset['test']
if USE_SMALL_DATASET:
    train_dataset = train_dataset.select(range(100)) # we use the first 100 entries to test the code
    val_dataset = val_dataset.select(range(100)) # we use the first 100 entries to test the code
    test_dataset = test_dataset.select(range(100)) # we use the first 100 entries to test the code
print(dataset['train'][0])
print(dataset['train'][0]['question'])
print(dataset['train'][0]['answer'])
print(dataset['test'][0])
print(dataset['test'][0]['question'])



DatasetDict({
    train: Dataset({
        features: ['qid', 'term', 'description', 'question', 'answer'],
        num_rows: 2821
    })
    test: Dataset({
        features: ['qid', 'term', 'description', 'question', 'answer'],
        num_rows: 490
    })
})
{'qid': '872', 'term': 'Swastika', 'description': 'a geometrical figure and an ancient religious icon in the cultures of Eurasia and 20th-century symbol of Nazism', 'question': 'Did the Hopi Indians use a symbol that was similar to the swastika?', 'answer': True}
Did the Hopi Indians use a symbol that was similar to the swastika?
True
{'qid': '564959490dd0b8316a88', 'term': None, 'description': None, 'question': 'can you use Microsoft Office without internet?', 'answer': None}
can you use Microsoft Office without internet?


In [44]:
# load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# tokenize the dataset
def tokenize_function(batch, tokenizer=tokenizer, field_name="question"):
    return tokenizer(batch[field_name], padding="max_length", truncation=True)


# load training dataset


# tokenize the datasets

tokenized_datasets["train"] = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets["val"] = val_dataset.map(tokenize_function, batched=True)
tokenized_datasets["test"] = test_dataset.map(tokenize_function, batched=True)

In [45]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\evaluate-cli.exe' -> 'c:\\Python312\\Scripts\\evaluate-cli.exe.deleteme'



In [ ]:
# check if GPU is available

import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # GPU name

False
0


AssertionError: Torch not compiled with CUDA enabled

In [48]:
import evaluate
# training
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

training_args = TrainingArguments("results", num_train_epochs=3, per_device_train_batch_size=8, per_device_eval_batch_size=8, logging_dir="logs", logging_steps=10)

metric = evaluate.load("accuracy")

trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["val"], compute_metrics=metric)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:
# Train the model
trainer.train()